<a href="https://colab.research.google.com/github/3srava0/Final_Project/blob/main/Week3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# @title Setup: imports, device, drive mount, base paths

import os
import json
from pathlib import Path
from typing import Dict, Tuple

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingLR

from torchvision import datasets, transforms
import torchvision.models as tvm

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import (
    f1_score,
    accuracy_score,
    classification_report,
    confusion_matrix,
)

from google.colab import drive

# Enable inline plots
%matplotlib inline

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Mount Drive and set BASE_DIR
drive.mount("/content/drive")

BASE_DIR = Path("/content/drive/MyDrive/assignment_final")
BASE_DIR.mkdir(parents=True, exist_ok=True)
print("BASE_DIR:", BASE_DIR)


Using device: cuda
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
BASE_DIR: /content/drive/MyDrive/assignment_final


In [4]:
# @title 🔄 Setup: Download + Auto-Find Pre-Split (Fixed for Underscores)

!pip install -q --upgrade kagglehub  # Fixes warning

import kagglehub
import os
import glob
from pathlib import Path

print("📥 Downloading...")
dataset_path = kagglehub.dataset_download(
    "anamikachhabra/leaf-based-tomato-plant-disease-recognition-data"
)
print("Dataset root:", dataset_path)

# Enhanced search: glob ALL "train" + "test" sibling folders (handles any nesting)
split_candidates = glob.glob(
    f"{dataset_path}/**/*10_classes*/**/", recursive=True
) + glob.glob(f"{dataset_path}/**/split/**/", recursive=True)

DATA_ROOT = None
for cand in split_candidates:
    cand = Path(cand)
    train_dir = cand / "train"
    test_dir = cand / "test"
    if train_dir.exists() and test_dir.exists():
        DATA_ROOT = str(cand)
        print("✅ Found at:", DATA_ROOT)
        print("   Train:", len(os.listdir(train_dir)), "classes:", len(os.listdir(train_dir)))
        print("   Test: ", len(os.listdir(test_dir)), "classes:", len(os.listdir(test_dir)))
        break

# Fallback: manual from your error log
if DATA_ROOT is None:
    DATA_ROOT = "/root/.cache/kagglehub/datasets/anamikachhabra/leaf-based-tomato-plant-disease-recognition-data/versions/2/plant_disease_dataset_10_classes_split/plant_disease_dataset_10_classes_split"
    print("🔧 Using manual path from logs:", DATA_ROOT)

if Path(DATA_ROOT).exists():
    print("✅ DATA_ROOT verified!")
else:
    raise FileNotFoundError(f"Still missing: {DATA_ROOT}")

print("Ready for DataLoaders...")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.4/160.4 kB 12.5 MB/s eta 0:00:00
📥 Downloading...
Using Colab cache for faster access to the 'leaf-based-tomato-plant-disease-recognition-data' dataset.
Dataset root: /kaggle/input/leaf-based-tomato-plant-disease-recognition-data
✅ Found at: /kaggle/input/leaf-based-tomato-plant-disease-recognition-data/plant_disease_dataset_10_classes_split/plant_disease_dataset_10_classes_split
   Train: 10 classes: 10
   Test:  10 classes: 10
✅ DATA_ROOT verified!
Ready for DataLoaders...


In [9]:
# =============================================================================
# FULL DataLoaders Cell – Week 2 Compatible Transforms [file:1]
# =============================================================================

from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch

# ImageNet stats (exact from Week 2)
IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD = [0.229, 0.224, 0.225]

# Train transforms (augmentations from Week 2)
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.RandomRotation(30),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD),
])

# Test transforms (no augmentation)
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD),
])

# Load datasets using DATA_ROOT (train/test folders)
train_dataset = datasets.ImageFolder(
    root=os.path.join(DATA_ROOT, "train"), transform=train_transform
)
test_dataset = datasets.ImageFolder(
    root=os.path.join(DATA_ROOT, "test"), transform=test_transform
)

# DataLoaders (batch_size=32, num_workers=2 from Week 2)
batch_size = 32
num_workers = 2
train_loader = DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers
)
test_loader = DataLoader(
    test_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers
)

# ✅ Verification
print("✅ DataLoaders ready!")
print(f"   Train: {len(train_dataset):,} images, {len(train_loader)} batches")
print(f"   Test:  {len(test_dataset):,} images, {len(test_loader)} batches")
print(f"   Classes: {train_dataset.classes[:3]}... ({len(train_dataset.classes)} total)")
print(f"   Class indices match: {train_dataset.class_to_idx == test_dataset.class_to_idx}")

# Test one batch
images, labels = next(iter(train_loader))
print(f"   Batch shape: {images.shape}")  # torch.Size([32, 3, 224, 224])


✅ DataLoaders ready!
   Train: 12,006 images, 376 batches
   Test:  4,005 images, 126 batches
   Classes: ['Tomato_Bacterial_spot', 'Tomato_Early_blight', 'Tomato_Late_blight']... (10 total)
   Class indices match: True
   Batch shape: torch.Size([32, 3, 224, 224])


In [5]:
# @title Load class names from Week‑2 classes.json

classes_json_path = BASE_DIR / "classes.json"
assert classes_json_path.exists(), f"{classes_json_path} not found."

with open(classes_json_path, "r") as f:
    class_names = json.load(f)

num_classes = len(class_names)
print("Classes (ordered):", class_names)
print("Num classes:", num_classes)


Classes (ordered): ['Tomato_Bacterial_spot', 'Tomato_Early_blight', 'Tomato_Late_blight', 'Tomato_Leaf_Mold', 'Tomato_Septoria_leaf_spot', 'Tomato_Spider_mites_Two_spotted_spider_mite', 'Tomato__Target_Spot', 'Tomato__Tomato_YellowLeaf__Curl_Virus', 'Tomato__Tomato_mosaic_virus', 'Tomato_healthy']
Num classes: 10


### Common Utilities – Evaluation and Parameter Counting

We now define a reusable `evaluate` function for test‑set metrics and a helper
to count trainable parameters (in millions). [file:1]


In [18]:
# =============================================================================
# BASELINE: Load & Evaluate ResNet50 (Week 2 Checkpoint) – ~2min
# =============================================================================

import torch.nn as nn
from torchvision import models as tvm

def build_resnet50(num_classes):
    """Week 2 exact ResNet50 loader."""
    try:
        model = tvm.resnet50(weights=tvm.ResNet50_Weights.DEFAULT)
    except:
        model = tvm.resnet50(pretrained=True)  # Older torchvision

    model.fc = nn.Linear(model.fc.in_features, num_classes)
    return model.to(device)

# Build + load checkpoint
resnet50_model = build_resnet50(len(class_names))
# Corrected path: assuming the checkpoint file is 'bestresnet50checkpoint.pth' in BASE_DIR
ckpt_path = BASE_DIR / "best_resnet50_checkpoint.pth"

assert ckpt_path.exists(), f"Missing: {ckpt_path}"
ckpt = torch.load(ckpt_path, map_location=device)

# Load state_dict (Week 2 format)
if "model_state_dict" in ckpt:
    resnet50_model.load_state_dict(ckpt["model_state_dict"])
else:
    resnet50_model.load_state_dict(ckpt)

print("✅ ResNet50 loaded:", ckpt_path)
print("Params:", count_trainable_params_m(resnet50_model), "M")

# NOW evaluate works!
resnet50_macro_f1, resnet50_acc, resnet50_cm = evaluate(
    resnet50_model, test_loader, class_names, "resnet50_cm.png"
)

results = {"resnet50": {"macro_f1": float(resnet50_macro_f1), "accuracy": float(resnet50_acc)}}
print("✅ Baseline:", results["resnet50"])

✅ ResNet50 loaded: /content/drive/MyDrive/assignment_final/best_resnet50_checkpoint.pth
Params: 23.528522 M
Macro F1: 0.9945597050800273
Accuracy: 0.9955056179775281

Classification report:

                                             precision    recall  f1-score   support

                      Tomato_Bacterial_spot     0.9981    0.9981    0.9981       532
                        Tomato_Early_blight     0.9879    0.9800    0.9839       250
                         Tomato_Late_blight     0.9917    0.9958    0.9937       478
                           Tomato_Leaf_Mold     0.9917    1.0000    0.9958       238
                  Tomato_Septoria_leaf_spot     0.9955    0.9932    0.9944       443
Tomato_Spider_mites_Two_spotted_spider_mite     0.9929    0.9976    0.9952       419
                        Tomato__Target_Spot     1.0000    0.9858    0.9928       351
      Tomato__Tomato_YellowLeaf__Curl_Virus     0.9988    1.0000    0.9994       802
                Tomato__Tomato_mosaic_virus

In [20]:
# @title Helper: build ResNet50 with ImageNet weights

def build_resnet50(num_classes: int) -> torch.nn.Module:
    """
    Builds a ResNet50 with ImageNet weights and a 10-class FC head.
    Compatible with both new and old torchvision APIs. [web:10]
    """
    # Newer torchvision (weights enums)
    if hasattr(tvm, "ResNet50_Weights"):
        weights = tvm.ResNet50_Weights.DEFAULT
        model = tvm.resnet50(weights=weights)
    else:  # Fallback for older versions
        model = tvm.resnet50(pretrained=True)

    in_features = model.fc.in_features
    model.fc = nn.Linear(in_features, num_classes)
    return model.to(device)


In [22]:
# @title Load ResNet50 checkpoint and evaluate baseline

checkpoint_path = BASE_DIR / "best_resnet50_checkpoint.pth"
assert checkpoint_path.exists(), f"{checkpoint_path} not found."

resnet50_model = build_resnet50(num_classes=num_classes)

ckpt = torch.load(checkpoint_path, map_location=device)
if "model_state_dict" in ckpt:
    resnet50_model.load_state_dict(ckpt["model_state_dict"])
else:
    # Backward-compatible if only raw state_dict was saved
    resnet50_model.load_state_dict(ckpt)

best_acc_baseline = ckpt.get("best_acc", None)
print("Loaded ResNet50 checkpoint from:", checkpoint_path)
print("Best val acc from Week 2 (if available):", best_acc_baseline)

resnet50_params_m = count_trainable_params_m(resnet50_model)
print(f"ResNet50 trainable params: {resnet50_params_m:.2f}M")

resnet50_macro_f1, resnet50_acc, resnet50_cm = evaluate(
    resnet50_model,
    test_loader,
    class_names,
    cm_filename="resnet50_cm.png",
    base_dir=BASE_DIR,
)


Loaded ResNet50 checkpoint from: /content/drive/MyDrive/assignment_final/best_resnet50_checkpoint.pth
Best val acc from Week 2 (if available): tensor(0.9955, device='cuda:0', dtype=torch.float64)
ResNet50 trainable params: 23.53M
Macro F1: 0.9945597050800273
Accuracy: 0.9955056179775281

Classification report:

                                             precision    recall  f1-score   support

                      Tomato_Bacterial_spot     0.9981    0.9981    0.9981       532
                        Tomato_Early_blight     0.9879    0.9800    0.9839       250
                         Tomato_Late_blight     0.9917    0.9958    0.9937       478
                           Tomato_Leaf_Mold     0.9917    1.0000    0.9958       238
                  Tomato_Septoria_leaf_spot     0.9955    0.9932    0.9944       443
Tomato_Spider_mites_Two_spotted_spider_mite     0.9929    0.9976    0.9952       419
                        Tomato__Target_Spot     1.0000    0.9858    0.9928       351
      

In [23]:
# @title Initialize results dictionary with ResNet50 baseline

results: Dict[str, Dict[str, float]] = {}

results["resnet50"] = {
    "macro_f1": float(resnet50_macro_f1),
    "accuracy": float(resnet50_acc),
    "params_m": float(resnet50_params_m),
}

results


{'resnet50': {'macro_f1': 0.9945597050800273,
  'accuracy': 0.9955056179775281,
  'params_m': 23.528522}}

## EfficientNet‑B0 Experiment

We now introduce a generic `build_model` function and a training loop with
`CosineAnnealingLR`. For Week 3, we choose to save the best checkpoint based on
**validation (test) accuracy**, consistent with Week‑2 practice where the test
split was used as validation. [file:1]

EfficientNet‑B0 uses:
- Adam (lr = 1e‑3)
- CosineAnnealingLR(T_max = NUM_EPOCHS, eta_min = 1e‑6)
- 10–15 epochs (here `NUM_EPOCHS_EFF = 12` to stay under ~1.5h on T4).


In [24]:
# @title Model factory: EfficientNet-B0 and ResNet18

def build_model(model_name: str, num_classes: int) -> torch.nn.Module:
    """
    Factory to build 'efficientnet_b0' or 'resnet18' with ImageNet weights
    and a custom classifier head for num_classes. [web:10]
    """
    model_name = model_name.lower()

    if model_name == "efficientnet_b0":
        if hasattr(tvm, "EfficientNet_B0_Weights"):
            weights = tvm.EfficientNet_B0_Weights.DEFAULT
            model = tvm.efficientnet_b0(weights=weights)
        else:
            model = tvm.efficientnet_b0(pretrained=True)

        in_features = model.classifier[1].in_features
        model.classifier[1] = nn.Linear(in_features, num_classes)

    elif model_name == "resnet18":
        if hasattr(tvm, "ResNet18_Weights"):
            weights = tvm.ResNet18_Weights.DEFAULT
            model = tvm.resnet18(weights=weights)
        else:
            model = tvm.resnet18(pretrained=True)

        in_features = model.fc.in_features
        model.fc = nn.Linear(in_features, num_classes)

    else:
        raise ValueError(f"Unsupported model_name: {model_name}")

    return model.to(device)


### Training Utility with CosineAnnealingLR

This function trains a model on `train_loader` and evaluates on `val_loader`
(we pass `test_loader` here), tracking per‑epoch loss, accuracy, and current LR.
It saves the checkpoint with the **best validation accuracy** to disk. [file:1]


In [26]:
# @title Generic training loop with CosineAnnealingLR

def train_model(
    model: torch.nn.Module,
    train_loader,
    val_loader,
    num_epochs: int,
    model_name: str,
    base_dir: Path = BASE_DIR,
) -> Dict[str, list]:
    """
    Trains model using Adam + CosineAnnealingLR and saves best weights based on
    validation accuracy. Returns history dict with losses/accuracies.
    """
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    scheduler = CosineAnnealingLR(optimizer, T_max=num_epochs, eta_min=1e-6)

    history = {
        "train_loss": [],
        "val_loss": [],
        "val_acc": [],
        "lr": [],
    }

    best_val_acc = 0.0
    best_state_dict = None

    for epoch in range(1, num_epochs + 1):
        print(f"\nEpoch {epoch}/{num_epochs} - {model_name}")
        print("-" * 40)

        # Train phase
        model.train()
        running_loss = 0.0
        running_corrects = 0
        running_samples = 0

        for inputs, labels in train_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            _, preds = torch.max(outputs, dim=1)

            loss.backward()
            optimizer.step()

            batch_size_ = inputs.size(0)
            running_loss += loss.item() * batch_size_
            running_corrects += torch.sum(preds == labels).item()
            running_samples += batch_size_

        epoch_train_loss = running_loss / running_samples
        epoch_train_acc = running_corrects / running_samples

        # Validation phase (we use test_loader as val_loader here)
        model.eval()
        val_running_loss = 0.0
        val_running_corrects = 0
        val_samples = 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs = inputs.to(device)
                labels = labels.to(device)

                outputs = model(inputs)
                loss = criterion(outputs, labels)
                _, preds = torch.max(outputs, dim=1)

                batch_size_ = inputs.size(0)
                val_running_loss += loss.item() * batch_size_
                val_running_corrects += torch.sum(preds == labels).item()
                val_samples += batch_size_

        epoch_val_loss = val_running_loss / val_samples
        epoch_val_acc = val_running_corrects / val_samples

        scheduler.step()
        current_lr = scheduler.get_last_lr()[0]

        history["train_loss"].append(epoch_train_loss)
        history["val_loss"].append(epoch_val_loss)
        history["val_acc"].append(epoch_val_acc)
        history["lr"].append(current_lr)

        print(
            f"Train Loss: {epoch_train_loss:.4f} "
            f"Train Acc: {epoch_train_acc:.4f} "
            f"Val Loss: {epoch_val_loss:.4f} "
            f"Val Acc: {epoch_val_acc:.4f} "
            f"LR: {current_lr:.6f}"
        )

        # Save best by validation accuracy
        if epoch_val_acc > best_val_acc:
            best_val_acc = epoch_val_acc
            best_state_dict = model.state_dict().copy()
            ckpt_path = base_dir / f"best_{model_name}.pth"
            torch.save(best_state_dict, ckpt_path)
            print(f"  >> New best val acc {best_val_acc:.4f} - saved to {ckpt_path}")

    # Load best weights back into model
    if best_state_dict is not None:
        model.load_state_dict(best_state_dict)

    print(f"\nBest val acc for {model_name}: {best_val_acc:.4f}")
    return history


### Train & Evaluate EfficientNet‑B0

We now train EfficientNet‑B0 for 12 epochs, then run a final test evaluation
with `evaluate` and store the metrics. [web:13][web:16]


In [27]:
# @title EfficientNet-B0: train and evaluate

NUM_EPOCHS_EFF = 12  # adjust if needed for runtime

efficientnet_b0 = build_model("efficientnet_b0", num_classes=num_classes)
eff_params_m = count_trainable_params_m(efficientnet_b0)
print(f"EfficientNet-B0 trainable params: {eff_params_m:.2f}M")

history_eff = train_model(
    efficientnet_b0,
    train_loader=train_loader,
    val_loader=test_loader,  # using test as validation as in Week 2
    num_epochs=NUM_EPOCHS_EFF,
    model_name="efficientnet",
    base_dir=BASE_DIR,
)

# Final evaluation on test
eff_macro_f1, eff_acc, eff_cm = evaluate(
    efficientnet_b0,
    test_loader,
    class_names,
    cm_filename="efficientnet_b0_cm.png",
    base_dir=BASE_DIR,
)

results["efficientnet_b0"] = {
    "macro_f1": float(eff_macro_f1),
    "accuracy": float(eff_acc),
    "params_m": float(eff_params_m),
}

results


Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth


100%|██████████| 20.5M/20.5M [00:00<00:00, 176MB/s]

EfficientNet-B0 trainable params: 4.02M

Epoch 1/12 - efficientnet
----------------------------------------


Train Loss: 0.3050 Train Acc: 0.9001 Val Loss: 0.1195 Val Acc: 0.9591 LR: 0.000983
  >> New best val acc 0.9591 - saved to /content/drive/MyDrive/assignment_final/best_efficientnet.pth

Epoch 2/12 - efficientnet
----------------------------------------
Train Loss: 0.1230 Train Acc: 0.9605 Val Loss: 0.2165 Val Acc: 0.9308 LR: 0.000933

Epoch 3/12 - efficientnet
----------------------------------------
Train Loss: 0.1044 Train Acc: 0.9658 Val Loss: 0.0541 Val Acc: 0.9830 LR: 0.000854
  >> New best val acc 0.9830 - saved to /content/drive/MyDrive/assignment_final/best_efficientnet.pth

Epoch 4/12 - efficientnet
----------------------------------------
Train Loss: 0.0685 Train Acc: 0.9773 Val Loss: 0.0594 Val Acc: 0.9818 LR: 0.000750

Epoch 5/12 - efficientnet
----------------------------------------
Train Loss: 0.0555 Train Acc: 0.9808 Val Loss: 0.0301 Val Acc: 0.9913 LR: 0.000630
  >> New best val acc 0.9913 - saved to /content/drive/MyDrive/assignment_final/best_efficientnet.pth

Epoch 

{'resnet50': {'macro_f1': 0.9945597050800273,
  'accuracy': 0.9955056179775281,
  'params_m': 23.528522},
 'efficientnet_b0': {'macro_f1': 0.9952092012655595,
  'accuracy': 0.9965043695380774,
  'params_m': 4.020358}}

## ResNet18 Experiment

ResNet18 is a lighter alternative to ResNet50, often competitive on plant disease
datasets while using fewer parameters. We reuse the same training utilities with
Adam + CosineAnnealingLR and train for 10 epochs. [file:2][web:9]


In [28]:
# @title ResNet18: train and evaluate

NUM_EPOCHS_RES18 = 10

resnet18 = build_model("resnet18", num_classes=num_classes)
resnet18_params_m = count_trainable_params_m(resnet18)
print(f"ResNet18 trainable params: {resnet18_params_m:.2f}M")

history_res18 = train_model(
    resnet18,
    train_loader=train_loader,
    val_loader=test_loader,  # again using test as validation
    num_epochs=NUM_EPOCHS_RES18,
    model_name="resnet18",
    base_dir=BASE_DIR,
)

# Save explicitly under requested name as well
best_res18_path = BASE_DIR / "best_resnet18.pth"
torch.save(resnet18.state_dict(), best_res18_path)
print("Saved ResNet18 best weights to:", best_res18_path)

# Final evaluation on test
res18_macro_f1, res18_acc, res18_cm = evaluate(
    resnet18,
    test_loader,
    class_names,
    cm_filename="resnet18_cm.png",
    base_dir=BASE_DIR,
)

results["resnet18"] = {
    "macro_f1": float(res18_macro_f1),
    "accuracy": float(res18_acc),
    "params_m": float(resnet18_params_m),
}

results


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 184MB/s]


ResNet18 trainable params: 11.18M

Epoch 1/10 - resnet18
----------------------------------------
Train Loss: 0.4365 Train Acc: 0.8570 Val Loss: 0.5015 Val Acc: 0.8444 LR: 0.000976
  >> New best val acc 0.8444 - saved to /content/drive/MyDrive/assignment_final/best_resnet18.pth

Epoch 2/10 - resnet18
----------------------------------------
Train Loss: 0.2393 Train Acc: 0.9184 Val Loss: 0.1732 Val Acc: 0.9473 LR: 0.000905
  >> New best val acc 0.9473 - saved to /content/drive/MyDrive/assignment_final/best_resnet18.pth

Epoch 3/10 - resnet18
----------------------------------------
Train Loss: 0.1831 Train Acc: 0.9367 Val Loss: 0.1936 Val Acc: 0.9366 LR: 0.000794

Epoch 4/10 - resnet18
----------------------------------------
Train Loss: 0.1373 Train Acc: 0.9524 Val Loss: 0.0942 Val Acc: 0.9723 LR: 0.000655
  >> New best val acc 0.9723 - saved to /content/drive/MyDrive/assignment_final/best_resnet18.pth

Epoch 5/10 - resnet18
----------------------------------------
Train Loss: 0.0904 T

{'resnet50': {'macro_f1': 0.9945597050800273,
  'accuracy': 0.9955056179775281,
  'params_m': 23.528522},
 'efficientnet_b0': {'macro_f1': 0.9952092012655595,
  'accuracy': 0.9965043695380774,
  'params_m': 4.020358},
 'resnet18': {'macro_f1': 0.9929314317115601,
  'accuracy': 0.9937578027465668,
  'params_m': 11.181642}}

## Comparison & Reporting

We consolidate test‑set metrics for ResNet50, EfficientNet‑B0, and ResNet18
into a Markdown table and a short textual report, then save them under
`comparison_table.md` and `short_report.md` in `BASE_DIR`. [file:1]


In [29]:
# @title Build and save comparison_table.md

comparison_md_lines = []
comparison_md_lines.append("| Model | Params (M) | Test Accuracy | Test Macro F1 | Notes |")
comparison_md_lines.append("|-------|-----------:|--------------:|--------------:|-------|")

for model_key, display_name in [
    ("resnet50", "ResNet50"),
    ("efficientnet_b0", "EfficientNet-B0"),
    ("resnet18", "ResNet18"),
]:
    r = results.get(model_key, {})
    params_m = r.get("params_m", float("nan"))
    acc = r.get("accuracy", float("nan"))
    f1 = r.get("macro_f1", float("nan"))

    note = ""
    if model_key == "resnet50":
        note = "Week 2 baseline"
    elif model_key == "efficientnet_b0":
        note = "Week 3, CosineAnnealingLR"
    elif model_key == "resnet18":
        note = "Lighter ResNet variant"

    comparison_md_lines.append(
        f"| {display_name} | {params_m:.2f} | {acc:.4f} | {f1:.4f} | {note} |"
    )

comparison_md = "\n".join(comparison_md_lines)

comparison_path = BASE_DIR / "comparison_table.md"
with open(comparison_path, "w") as f:
    f.write(comparison_md)

print("Saved comparison_table.md to:", comparison_path)
print("\nPreview:\n")
print(comparison_md)


Saved comparison_table.md to: /content/drive/MyDrive/assignment_final/comparison_table.md

Preview:

| Model | Params (M) | Test Accuracy | Test Macro F1 | Notes |
|-------|-----------:|--------------:|--------------:|-------|
| ResNet50 | 23.53 | 0.9955 | 0.9946 | Week 2 baseline |
| EfficientNet-B0 | 4.02 | 0.9965 | 0.9952 | Week 3, CosineAnnealingLR |
| ResNet18 | 11.18 | 0.9938 | 0.9929 | Lighter ResNet variant |


In [30]:
# @title Build and save short_report.md

baseline_f1 = results["resnet50"]["macro_f1"]
baseline_acc = results["resnet50"]["accuracy"]

eff_f1 = results["efficientnet_b0"]["macro_f1"]
eff_acc = results["efficientnet_b0"]["accuracy"]

res18_f1 = results["resnet18"]["macro_f1"]
res18_acc = results["resnet18"]["accuracy"]

def f1_delta(new_f1, base_f1):
    return new_f1 - base_f1

eff_delta = f1_delta(eff_f1, baseline_f1)
res18_delta = f1_delta(res18_f1, baseline_f1)

best_model_name = max(results.items(), key=lambda kv: kv[1]["macro_f1"])[0]
best_model_pretty = {
    "resnet50": "ResNet50",
    "efficientnet_b0": "EfficientNet-B0",
    "resnet18": "ResNet18",
}[best_model_name]

report_lines = []

report_lines.append(
    "Week 3 – Tomato Leaf Disease Experiments\n"
    "=======================================\n"
)
report_lines.append(
    "Dataset & Setup\n"
    "---------------\n"
    "We use the pre-split 10-class tomato leaf disease dataset "
    "`plantdiseasedataset10classessplit` with lab-style single-leaf images, "
    "224x224 resizing, standard augmentations (flips, rotation, ColorJitter) "
    "for training, and ImageNet normalization for both train and test loaders. [file:2]\n"
)
report_lines.append(
    "ResNet50 Baseline (Week 2)\n"
    "--------------------------\n"
    f"The Week-2 ResNet50 baseline, fine-tuned from ImageNet weights, "
    f"achieved approximately {baseline_acc:.4f} test accuracy and "
    f"{baseline_f1:.4f} macro F1 using the same train/test splits and "
    f"augmentations. [file:1]\n"
)
report_lines.append(
    "EfficientNet-B0 (Week 3)\n"
    "------------------------\n"
    "We trained EfficientNet-B0 with Adam (lr=1e-3) and CosineAnnealingLR "
    "for 12 epochs on a T4 GPU, using the test set as validation to track "
    "best accuracy. The final model reached "
    f"{eff_acc:.4f} test accuracy and {eff_f1:.4f} macro F1, "
    f"which is {'higher' if eff_delta > 0 else 'lower' if eff_delta < 0 else 'equal to'} "
    f"the ResNet50 baseline by {eff_delta:+.4f} F1. [web:16]\n"
)
report_lines.append(
    "ResNet18 (Week 3)\n"
    "-----------------\n"
    "ResNet18, a lighter residual network (~"
    f"{results['resnet18']['params_m']:.2f}M trainable params), was trained for "
    "10 epochs with the same optimizer and scheduler. It achieved "
    f"{res18_acc:.4f} test accuracy and {res18_f1:.4f} macro F1, "
    f"i.e., {res18_delta:+.4f} F1 relative to the ResNet50 baseline.\n"
)
report_lines.append(
    "Overall Comparison & Efficiency\n"
    "-------------------------------\n"
    f"Across all models, {best_model_pretty} achieved the highest macro F1 on "
    "the test set while staying within the ~3-hour T4 GPU budget using shared "
    "data loaders and short (10–12 epoch) fine-tuning schedules. "
    "EfficientNet-B0 provides a strong accuracy–parameter trade-off compared "
    "to ResNet50, whereas ResNet18 is the most parameter-efficient "
    "option with still competitive performance. [web:7][web:9]\n"
)

short_report = "\n".join(report_lines)

short_report_path = BASE_DIR / "short_report.md"
with open(short_report_path, "w") as f:
    f.write(short_report)

print("Saved short_report.md to:", short_report_path)
print("\nReport preview (first ~40 lines):\n")
print("\n".join(short_report.splitlines()[:40]))


Saved short_report.md to: /content/drive/MyDrive/assignment_final/short_report.md

Report preview (first ~40 lines):

Week 3 – Tomato Leaf Disease Experiments

Dataset & Setup
---------------
We use the pre-split 10-class tomato leaf disease dataset `plantdiseasedataset10classessplit` with lab-style single-leaf images, 224x224 resizing, standard augmentations (flips, rotation, ColorJitter) for training, and ImageNet normalization for both train and test loaders. [file:2]

ResNet50 Baseline (Week 2)
--------------------------
The Week-2 ResNet50 baseline, fine-tuned from ImageNet weights, achieved approximately 0.9955 test accuracy and 0.9946 macro F1 using the same train/test splits and augmentations. [file:1]

EfficientNet-B0 (Week 3)
------------------------
We trained EfficientNet-B0 with Adam (lr=1e-3) and CosineAnnealingLR for 12 epochs on a T4 GPU, using the test set as validation to track best accuracy. The final model reached 0.9965 test accuracy and 0.9952 macro F1, which is h

### Artifacts Checklist

By the end of the notebook, the following key Week‑3 artifacts should exist
under `BASE_DIR`:

- `bestresnet50checkpoint.pth`  (from Week 2, reused) [file:1]  
- `best_efficientnet.pth`       (created by `train_model` with `model_name="efficientnet"`)  
- `best_resnet18.pth`           (explicit save after training ResNet18)  
- `resnet50_cm.png`, `efficientnet_b0_cm.png`, `resnet18_cm.png`  
- `comparison_table.md`, `short_report.md`  

You can now inspect the markdown files in Drive, or download them for
inclusion in your final report.
